In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from io import BytesIO

!pip install azure-storage-blob
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
#defining azure connection string, container name & blob name
CONNECTION_STRING_AZURE_STORAGE='DefaultEndpointsProtocol=https;AccountName=affordablehousing;AccountKey=J+3/tpTlA4eRSpu7TOQ+zSx7jErO15HRSGZa/e2iLxVDiFzjMP9njTT71S5W284vk4EjNA4i36+y+AStS5uxew==;EndpointSuffix=core.windows.net'
CONTAINER_AZURE='affordablehousing'

#creating azure functions to upload to the blob
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")
    
#creating azure functions 
def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [3]:
#downloading csv from azure storage
download_data=azure_download_blob(CONNECTION_STRING_AZURE_STORAGE, CONTAINER_AZURE, 'raw_affordablehousingdata.csv')

In [4]:
#converting csv download into df
download_data=pd.read_csv(BytesIO(download_data))
download_data

,project_id,project_name,project_start_date,project_completion_date,building_id,house_number,street_name,borough,postcode,bbl,...,_2_br_units,_3_br_units,_4_br_units,_5_br_units,_6_br_units,unknown_br_units,counted_rental_units,counted_homeownership_units,all_counted_units,total_units
0,44218,MEC E. 125TH ST. PARCEL B WEST,2018-12-31T00:00:00.000,2022-02-16T00:00:00.000,987329.0,2319,3 AVENUE,Manhattan,10035.0,1.017908e+09,...,129,15,0,0,0,0,297,0,297,404
1,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,927737.0,335,RALPH AVENUE,Brooklyn,11233.0,3.015560e+09,...,11,0,0,0,0,0,0,13,13,13
2,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,969695.0,35,ROCHESTER AVENUE,Brooklyn,11233.0,3.017090e+09,...,6,0,0,0,0,0,0,8,8,8
3,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,975702.0,18-22,SUYDAM PLACE,Brooklyn,11233.0,3.017090e+09,...,1,0,0,0,0,0,0,15,15,15
4,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,977564.0,329,RALPH AVENUE,Brooklyn,11233.0,3.015560e+09,...,7,0,0,0,0,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7248,74997,1819 SEDGWICK AVENUE,2023-11-02T00:00:00.000,NaN,108437.0,1819,SEDGWICK AVENUE,Bronx,10453.0,2.028810e+09,...,0,1,0,0,0,0,7,0,7,8
7249,74998,49 BORINQUEN PLACE,2023-10-31T00:00:00.000,NaN,1011516.0,49,BORINQUEN PLACE,Brooklyn,11211.0,NaN,...,5,0,0,0,0,0,5,0,5,16
7250,74999,115 WEST 190 STREET APARTMENTS,2023-10-30T00:00:00.000,NaN,121294.0,115,WEST 190 STREET,Bronx,10468.0,2.032200e+09,...,3,0,0,0,0,0,12,0,12,37
7251,75011,VITAL BK UTICA CRESCENT,2023-12-14T00:00:00.000,NaN,1011557.0,564,UTICA AVENUE,Brooklyn,11203.0,NaN,...,78,20,0,0,0,2,322,0,322,322


In [5]:
download_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7253 entries, 0 to 7252
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     7253 non-null   int64  
 1   project_name                   7253 non-null   object 
 2   project_start_date             7253 non-null   object 
 3   project_completion_date        5486 non-null   object 
 4   building_id                    5796 non-null   float64
 5   house_number                   7253 non-null   object 
 6   street_name                    7253 non-null   object 
 7   borough                        7253 non-null   object 
 8   postcode                       5745 non-null   float64
 9   bbl                            5677 non-null   float64
 10  bin                            5677 non-null   float64
 11  community_board                7253 non-null   object 
 12  council_district               7253 non-null   i

In [6]:
#dropping census_tract since all null values
download_data=download_data.drop(columns=['census_tract'])
print(download_data.columns)

Index(['project_id', 'project_name', 'project_start_date',
       'project_completion_date', 'building_id', 'house_number', 'street_name',
       'borough', 'postcode', 'bbl', 'bin', 'community_board',
       'council_district', 'neighborhood_tabulation_area', 'latitude',
       'longitude', 'latitude_internal', 'longitude_internal',
       'building_completion_date', 'reporting_construction_type',
       'extended_affordability_status', 'prevailing_wage_status',
       'extremely_low_income_units', 'very_low_income_units',
       'low_income_units', 'moderate_income_units', 'middle_income_units',
       'other_income_units', 'studio_units', '_1_br_units', '_2_br_units',
       '_3_br_units', '_4_br_units', '_5_br_units', '_6_br_units',
       'unknown_br_units', 'counted_rental_units',
       'counted_homeownership_units', 'all_counted_units', 'total_units'],
      dtype='object')


In [7]:
#removing any duplicate building_id values
download_data=download_data.drop_duplicates(subset=['building_id'])
download_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5739 entries, 0 to 7252
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     5739 non-null   int64  
 1   project_name                   5739 non-null   object 
 2   project_start_date             5739 non-null   object 
 3   project_completion_date        4059 non-null   object 
 4   building_id                    5738 non-null   float64
 5   house_number                   5739 non-null   object 
 6   street_name                    5739 non-null   object 
 7   borough                        5739 non-null   object 
 8   postcode                       5687 non-null   float64
 9   bbl                            5619 non-null   float64
 10  bin                            5619 non-null   float64
 11  community_board                5739 non-null   object 
 12  council_district               5739 non-null   int64 

In [8]:
#dropping null values in building_id 
download_data=download_data.dropna(subset=['building_id'])
download_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5738 entries, 0 to 7252
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     5738 non-null   int64  
 1   project_name                   5738 non-null   object 
 2   project_start_date             5738 non-null   object 
 3   project_completion_date        4058 non-null   object 
 4   building_id                    5738 non-null   float64
 5   house_number                   5738 non-null   object 
 6   street_name                    5738 non-null   object 
 7   borough                        5738 non-null   object 
 8   postcode                       5687 non-null   float64
 9   bbl                            5619 non-null   float64
 10  bin                            5619 non-null   float64
 11  community_board                5738 non-null   object 
 12  council_district               5738 non-null   int64 

In [9]:
#allows to write over copied dataframe
pd.options.mode.copy_on_write = True

In [10]:
#converting building_id from float to int 
download_data['building_id']=download_data['building_id'].astype(int)
download_data

,project_id,project_name,project_start_date,project_completion_date,building_id,house_number,street_name,borough,postcode,bbl,...,_2_br_units,_3_br_units,_4_br_units,_5_br_units,_6_br_units,unknown_br_units,counted_rental_units,counted_homeownership_units,all_counted_units,total_units
0,44218,MEC E. 125TH ST. PARCEL B WEST,2018-12-31T00:00:00.000,2022-02-16T00:00:00.000,987329,2319,3 AVENUE,Manhattan,10035.0,1.017908e+09,...,129,15,0,0,0,0,297,0,297,404
1,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,927737,335,RALPH AVENUE,Brooklyn,11233.0,3.015560e+09,...,11,0,0,0,0,0,0,13,13,13
2,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,969695,35,ROCHESTER AVENUE,Brooklyn,11233.0,3.017090e+09,...,6,0,0,0,0,0,0,8,8,8
3,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,975702,18-22,SUYDAM PLACE,Brooklyn,11233.0,3.017090e+09,...,1,0,0,0,0,0,0,15,15,15
4,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,NaN,977564,329,RALPH AVENUE,Brooklyn,11233.0,3.015560e+09,...,7,0,0,0,0,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7248,74997,1819 SEDGWICK AVENUE,2023-11-02T00:00:00.000,NaN,108437,1819,SEDGWICK AVENUE,Bronx,10453.0,2.028810e+09,...,0,1,0,0,0,0,7,0,7,8
7249,74998,49 BORINQUEN PLACE,2023-10-31T00:00:00.000,NaN,1011516,49,BORINQUEN PLACE,Brooklyn,11211.0,NaN,...,5,0,0,0,0,0,5,0,5,16
7250,74999,115 WEST 190 STREET APARTMENTS,2023-10-30T00:00:00.000,NaN,121294,115,WEST 190 STREET,Bronx,10468.0,2.032200e+09,...,3,0,0,0,0,0,12,0,12,37
7251,75011,VITAL BK UTICA CRESCENT,2023-12-14T00:00:00.000,NaN,1011557,564,UTICA AVENUE,Brooklyn,11203.0,NaN,...,78,20,0,0,0,2,322,0,322,322


In [11]:
#dropping rows with null values in building_completion_date since we are only interested in completed buildings for analysis
download_data=download_data.dropna(subset=['building_completion_date'])
download_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4282 entries, 0 to 7160
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     4282 non-null   int64  
 1   project_name                   4282 non-null   object 
 2   project_start_date             4282 non-null   object 
 3   project_completion_date        4058 non-null   object 
 4   building_id                    4282 non-null   int64  
 5   house_number                   4282 non-null   object 
 6   street_name                    4282 non-null   object 
 7   borough                        4282 non-null   object 
 8   postcode                       4245 non-null   float64
 9   bbl                            4218 non-null   float64
 10  bin                            4218 non-null   float64
 11  community_board                4282 non-null   object 
 12  council_district               4282 non-null   int64 

In [12]:
#dropping rows with null values in project_completion_date and project_start_date since we are only interested in completed projects for analysis
download_data=download_data.dropna(subset=['project_completion_date','project_start_date'])
download_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4058 entries, 0 to 7160
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     4058 non-null   int64  
 1   project_name                   4058 non-null   object 
 2   project_start_date             4058 non-null   object 
 3   project_completion_date        4058 non-null   object 
 4   building_id                    4058 non-null   int64  
 5   house_number                   4058 non-null   object 
 6   street_name                    4058 non-null   object 
 7   borough                        4058 non-null   object 
 8   postcode                       4022 non-null   float64
 9   bbl                            3995 non-null   float64
 10  bin                            3995 non-null   float64
 11  community_board                4058 non-null   object 
 12  council_district               4058 non-null   int64 

In [13]:
#creating new column for total_low_income_units
download_data['total_low_income_units']=download_data['extremely_low_income_units']+download_data['very_low_income_units']+download_data['low_income_units']
download_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4058 entries, 0 to 7160
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     4058 non-null   int64  
 1   project_name                   4058 non-null   object 
 2   project_start_date             4058 non-null   object 
 3   project_completion_date        4058 non-null   object 
 4   building_id                    4058 non-null   int64  
 5   house_number                   4058 non-null   object 
 6   street_name                    4058 non-null   object 
 7   borough                        4058 non-null   object 
 8   postcode                       4022 non-null   float64
 9   bbl                            3995 non-null   float64
 10  bin                            3995 non-null   float64
 11  community_board                4058 non-null   object 
 12  council_district               4058 non-null   int64 

In [14]:
#creating new column for total_non-low_income_units
download_data['total_non_low_income_units']=download_data['moderate_income_units']+download_data['middle_income_units']
download_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4058 entries, 0 to 7160
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   project_id                     4058 non-null   int64  
 1   project_name                   4058 non-null   object 
 2   project_start_date             4058 non-null   object 
 3   project_completion_date        4058 non-null   object 
 4   building_id                    4058 non-null   int64  
 5   house_number                   4058 non-null   object 
 6   street_name                    4058 non-null   object 
 7   borough                        4058 non-null   object 
 8   postcode                       4022 non-null   float64
 9   bbl                            3995 non-null   float64
 10  bin                            3995 non-null   float64
 11  community_board                4058 non-null   object 
 12  council_district               4058 non-null   int64 

In [ ]:
#dropping columns not needed for analysis, these columns are dropped because I will only focus on low income vs non-low income per location 
download_data=download_data.drop(columns=['project_start_date','project_completion_date','bbl','bin','community_board','council_district','neighborhood_tabulation_area',
                                          'latitude_internal','longitude_internal','building_completion_date','extended_affordability_status','prevailing_wage_status',
                                            'other_income_units','unknown_br_units'])
download_data.info()

In [ ]:
# convert dataframe to csv
output = StringIO()
download_data.to_csv(output, index=False)
data = output.getvalue()
output.close()
blob_name = "cleaned_affordablehousingdata.csv"

In [ ]:
#blob client creation
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

#uploading csv data
blob_client.upload_blob(data, overwrite=True)